In [3]:
import sys
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics


movie_reviews_data_folder = r'.\movie_review'
dataset = load_files(movie_reviews_data_folder, shuffle=False,encoding='utf8')
# print("n_samples: %d" % len(dataset.data))
RANDOM_STATE=940302
docs_train, docs_test, y_train, y_test = train_test_split(
    dataset.data, dataset.target, test_size=0.2, random_state=RANDOM_STATE)
from sklearn.linear_model import SGDClassifier

# clf_pipe = Pipeline([
#     ('vect',CountVectorizer()),
#     ('tfidf',TfidfTransformer()),
#     ('clf',SGDClassifier())])

# clf_pipe.fit(docs_train,y_train)
# y_pre = clf_pipe.predict(docs_test)
# import numpy as np
# np.mean(y_pre==y_test)


In [67]:
# TASK: Build a grid search to find out whether unigrams or bigrams are
# more useful.
# Fit the pipeline on the training set using grid search for the parameters
from sklearn.naive_bayes import MultinomialNB
pipe_clf = Pipeline([('step1',TfidfVectorizer()), ('step2', MultinomialNB())])
params={'step1__ngram_range':[(1,1),(1,2)],
       'step2__alpha':(1e-1,1e-2,1e-3),
       }
gds = GridSearchCV(estimator=pipe_clf,param_grid=params)
gds = gds.fit(docs_train,y_train)
print(gds.cv_results_)
print('==========================')
print('最佳分数：{}'.format(gds.best_score_))                                  
for param_name in sorted(params.keys()):
    print("%s: %r" % (param_name, gds.best_params_[param_name]))

{'mean_fit_time': array([ 0.6846149 ,  0.65458536,  0.66853476,  2.96273073,  3.08042979,
        2.98036615]), 'std_fit_time': array([ 0.03108948,  0.00376383,  0.00857129,  0.1027057 ,  0.1181902 ,
        0.01864673]), 'mean_score_time': array([ 0.30867227,  0.28258952,  0.27658073,  0.72772272,  0.72240257,
        0.70943634]), 'std_score_time': array([ 0.02721284,  0.00892447,  0.00571772,  0.01502258,  0.01356086,
        0.02194635]), 'param_step1__ngram_range': masked_array(data = [(1, 1) (1, 1) (1, 1) (1, 2) (1, 2) (1, 2)],
             mask = [False False False False False False],
       fill_value = ?)
, 'param_step2__alpha': masked_array(data = [0.1 0.01 0.001 0.1 0.01 0.001],
             mask = [False False False False False False],
       fill_value = ?)
, 'params': [{'step1__ngram_range': (1, 1), 'step2__alpha': 0.1}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.01}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.001}, {'step1__ngram_range': (1, 2), 'step2__alpha

In [71]:
# TASK: Predict the outcome on the testing set and store it in a variable
# named y_predicted
y_predicted = gds.predict(docs_test)
print('混淆矩阵：\n{}'.format(metrics.confusion_matrix(y_predicted,y_test)))
# Print the classification report
print('---------------------------------------------------------------')
def pipecv_report(clf,params):
    print(clf.cv_results_)
    print('==========================')
    print('最佳分数：{}'.format(clf.best_score_))                                  
    for param_name in sorted(params.keys()):
        print("%s: %r" % (param_name, clf.best_params_[param_name]))
        
pipecv_report(gds,params)

混淆矩阵：
[[155  35]
 [ 37 173]]
---------------------------------------------------------------
{'mean_fit_time': array([ 0.6846149 ,  0.65458536,  0.66853476,  2.96273073,  3.08042979,
        2.98036615]), 'std_fit_time': array([ 0.03108948,  0.00376383,  0.00857129,  0.1027057 ,  0.1181902 ,
        0.01864673]), 'mean_score_time': array([ 0.30867227,  0.28258952,  0.27658073,  0.72772272,  0.72240257,
        0.70943634]), 'std_score_time': array([ 0.02721284,  0.00892447,  0.00571772,  0.01502258,  0.01356086,
        0.02194635]), 'param_step1__ngram_range': masked_array(data = [(1, 1) (1, 1) (1, 1) (1, 2) (1, 2) (1, 2)],
             mask = [False False False False False False],
       fill_value = ?)
, 'param_step2__alpha': masked_array(data = [0.1 0.01 0.001 0.1 0.01 0.001],
             mask = [False False False False False False],
       fill_value = ?)
, 'params': [{'step1__ngram_range': (1, 1), 'step2__alpha': 0.1}, {'step1__ngram_range': (1, 1), 'step2__alpha': 0.01}, {'step

In [4]:
#运行pipe2 要reload dataset
pipe_2 = Pipeline([('step1',TfidfVectorizer()), ('step2', SGDClassifier(random_state=940302,
                                           max_iter=200, tol=None))])
pipe_2_params={'step1__ngram_range':[(1,1),(1,2)],
                'step2__alpha': (1e-4,1e-5),
              'step2__loss':('hinge','modified_huber'),
              'step2__penalty':('l1','l2'),    
       }
gds_2 = GridSearchCV(estimator=pipe_2,param_grid=pipe_2_params)
gds_2 = gds_2.fit(docs_train,y_train)
print(gds_2.cv_results_)
print('==========================')


{'mean_fit_time': array([ 1.53638665,  0.84108337,  1.50498199,  0.92185648,  1.52556356,
        0.82376011,  1.64792975,  0.8337481 ,  6.36994457,  3.83574422,
        6.57771293,  4.17647401,  6.18644985,  3.82011867,  6.60001938,
        3.88361549]), 'std_fit_time': array([ 0.13098273,  0.00402166,  0.01310658,  0.01742596,  0.00862673,
        0.01572789,  0.01140954,  0.01976063,  0.02655417,  0.03316774,
        0.04260522,  0.02093559,  0.02895938,  0.04493861,  0.03124613,
        0.01628592]), 'mean_score_time': array([ 0.30960663,  0.30783129,  0.31347855,  0.30852087,  0.31317782,
        0.30119602,  0.30119292,  0.29755076,  0.74135272,  0.73238977,
        0.70478511,  0.70545093,  0.70644848,  0.72674378,  0.73271187,
        0.71177681]), 'std_score_time': array([ 0.00493704,  0.01222322,  0.00576236,  0.0094468 ,  0.01078897,
        0.00705274,  0.00895726,  0.0061035 ,  0.05998034,  0.0237488 ,
        0.01506885,  0.02044336,  0.01439122,  0.01024166,  0.00541939,

In [9]:
print('最佳分数：{}'.format(gds_2.best_score_))                                  
for param_name in sorted(pipe_2_params.keys()):
    print("%s: %r" % (param_name, gds_2.best_params_[param_name]))
y_predicted = gds_2.predict(docs_test)
print(metrics.classification_report(y_test, y_predicted,target_names=dataset.target_names))

最佳分数：0.835
step1__ngram_range: (1, 2)
step2__alpha: 1e-05
step2__loss: 'hinge'
step2__penalty: 'l2'
             precision    recall  f1-score   support

        neg       0.87      0.76      0.81       192
        pos       0.80      0.89      0.84       208

avg / total       0.83      0.83      0.83       400

